# Combining all together

Now we have all the pieces necessary to make implement the interactive optimization process using clustering as surrogates and different scalarization functions. Yeah.

In [1]:
 %matplotlib inline
import seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from ASF import ASF
from gradutil import *
from pyomo.opt import SolverFactory
seedn = 1

First lets take all the data in

In [2]:
%%time
revenue, carbon, deadwood, ha = init_boreal()
n_revenue = nan_to_bau(revenue)
n_carbon= nan_to_bau(carbon)
n_deadwood = nan_to_bau(deadwood)
n_ha = nan_to_bau(ha)
ide = ideal(False)
nad = nadir(False)
opt = SolverFactory('glpk')

CPU times: user 53.9 s, sys: 336 ms, total: 54.3 s
Wall time: 54.6 s


In [3]:
x = pd.concat((n_revenue, n_carbon, n_deadwood, n_ha), axis=1)
x_stack = np.dstack((n_revenue, n_carbon, n_deadwood, n_ha))

Normalize all the columns in 0-1 scale

In [4]:
%%time
x_norm = normalize(x.values)
x_norm_stack = normalize(x_stack)

CPU times: user 68 ms, sys: 8 ms, total: 76 ms
Wall time: 75 ms


/home/jousi/Documents/gradujose/code/gradutil.py:59: RuntimeWarning: invalid value encountered in true_divide
  norm_data = np.where(normax != 0., norm_data / normax, 0)


Cluster the data to some clusters and calculate correponding weights

In [5]:
%%time 
nclust1 = 50
c, xtoc, dist = cluster(x_norm, nclust1, seedn, verbose=1)
w = np.array([sum(xtoc == i) for i in range(len(c))])

kmeans: X (29666, 28)  centres (50, 28)  delta=0.0001  maxiter=100  metric=cosine
kmeans: 20 iterations  cluster sizes: array([ 815,  671,  790,  501,  509,  312,  640,  848,  426, 1122,  358,
        896,  307,  357,  884,  317,  392, 1051,  780,  791,  766,  216,
        453,  683,   86,  512,  917, 1081,  687,   74,  323,  409,  829,
        259,  378, 1033,  577,  635,  635,  828,  386,  454,  566,  478,
        664,  259,  760,  120,  918,  913])
CPU times: user 9.54 s, sys: 16 ms, total: 9.56 s
Wall time: 9.56 s


Calculate new cluster centers using average from normalized data

In [6]:
c_new = np.array([x_norm_stack[xtoc == i].mean(axis=0) for i in range(nclust1)])

In [7]:
%%time
ref = np.array((ide[0], 0, 0, 0))
asf = ASF(ide, nad, ref, c_new, weights=w)
opt.solve(asf.model)

CPU times: user 300 ms, sys: 8 ms, total: 308 ms
Wall time: 323 ms


In [47]:
model_to_real_values(x_stack, xtoc, asf.model)

array([  1.26427443e+08,   3.89994420e+06,   1.80326542e+05,
         1.85847006e+04])

## Trying to calculate ideal and nadir using clustering

In [48]:
c_new_unscale = np.array([x_stack[xtoc == i].mean(axis=0) for i in range(nclust1)])

In [49]:
data = c_new
weights = w
solver = SolverFactory('glpk')
problems = []
for i in range(np.shape(data)[-1]):
    problems.append(BorealWeightedProblem(data[:, :, i], weights))
for j in range(len(problems)):
    solver.solve(problems[j].model)

In [57]:
payoff = [[np.sum(cluster_to_value(c_new_unscale[:, :, i], res_to_list(problems[j].model), weights))
                   for i in range(np.shape(data)[-1])]
                  for j in range(len(problems))]
ideal = np.max(payoff, axis=0)
nadir = np.min(payoff, axis=0)

In [59]:
ideal, nadir

(array([  1.85745518e+08,   4.22843891e+06,   1.95555464e+05,
          1.85860769e+04]),
 array([  9.41298716e+07,   3.75198024e+06,   8.17197798e+04,
          1.27816665e+04]))

In [60]:
ide, nad

(array([  2.49966739e+08,   4.44900147e+06,   2.18153215e+05,
          2.02252577e+04]),
 array([  3.09084573e+07,   2.83139915e+06,   8.02116726e+04,
          1.19880519e+04]))

## Verifying


In [63]:
data = x_stack
weights = np.ones(len(x_stack))
solver = SolverFactory('glpk')
problems = []
for i in range(np.shape(data)[-1]):
    problems.append(BorealWeightedProblem(data[:, :, i], weights))
for j in range(len(problems)):
    solver.solve(problems[j].model)

In [65]:
payoff = [[np.sum(cluster_to_value(x_stack[:, :, i], res_to_list(problems[j].model), weights))
                   for i in range(np.shape(data)[-1])]
                  for j in range(len(problems))]
ideal = np.max(payoff, axis=0)
nadir = np.min(payoff, axis=0)

In [67]:
ideal-ide, nadir-nad

(array([ 0.,  0.,  0.,  0.]),
 array([  1.40000209e-02,  -4.99996822e-04,  -1.08652166e-05,
          4.10918292e-05]))

So it looks like that the calculating of ideal and nadir is possible using this method